# Genotype-Phenotype Converters
This file creates the code that allows us to convert back and forth between a bird's genotype and its phenotype

It also contains **Technical Note** sections throughout, that explain the coding and tech details of the code, and give links to external resources

## Save Genotype
First, we create a function that lets us take a dictionary of genes in the bird, and add or update the genotype for a specific gene

**Technical Note:** [A dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) is a type of data in Python that contains a bunch of pieces of smaller data linked to specific words. Data can be saved under a specific name, and then retrived by using that same name, much like definitions in a paper dictionary.

In [35]:
def saveGenotypeToBird(bird, geneName, genotype):
    if genotype == 'WT/WT' or genotype == 'Z(WT)/Z(WT)' or genotype == 'Z(WT)/w':
        if geneName in bird:
            del bird[geneName]
    else:
        bird[geneName] = genotype

### saveGenotypeToBird Tests
We can test this code by adding a new gene to a bird, and seeing that the genotype is updated.

**Technical Note:** This block of code is called a unit test, and is used to verify that the main code actually does what we want it to do.  The comments are formatted in [Gherkin](https://cucumber.io/docs/gherkin/), which is a language that is used to describe code tests in a human-readable way

**Technical Note:** If the tests pass, the code block with the tests will not produce any output, so that when we reuse these functions in another notebook, we don't change the UI.  If the tests fail, then the code block will produce errors.

In [36]:
# GIVEN an empty bird genome
bird = {}
# WHEN saveGenotypeToBird is called with a "Fake Gene" genotype of "WT/WT"
saveGenotypeToBird(bird, "Fake Gene", "WT/WT")
# THEN the bird will not contain a "WT/WT" genotype for "Fake Gene"
if not bird == {}:
    raise Exception('Failed to verify genotype converter - Issue with adding wild type gene to empty bird')

In [3]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'WT/WT'}
# WHEN saveGenotypeToBird is called with a "Fake Gene" genotype of "f/f"
saveGenotypeToBird(bird, "Fake Gene", "f/f")
# THEN the bird will contain a "f/f" genotype for "Fake Gene"
if not bird == {'Fake Gene': 'f/f'}:
    raise Exception('Failed to verify genotype converter - Issue with updating genotype')

## Save Phenotype
Next, we need to create a function that takes a phenotype and saves it as a genotype.  We want to make sure to reset any genotype data that conflicts with the new phenotype, and handle cases where the phenotype may require more complex genotypes. We will break this down into several smaller functions, and link them all up together to form the final function that the peafowl calculator will actually use

### Modify Simple Genes
The first step in saving the phenotype will be to update the genotype for all the autosomal genes that only have two allotypes - wild type or a single possible trait.  To do this we will check each simple gene one-at-a-time.  If the name of the simple gene matches the given phenotype, we will update the genotype for that gene to be homozygous for the trait. However, if the name of the simple gene does NOT match the phenotype, we will delete any existing genotype for that gene, so it does not conflict with the given phenotype.

(Don't worry about phenotypes that require multiple simple genes to be expressed; we will handle those in another function, and possibly deleting the genotypes that might be used in them will not prevent those genotypes from being re-updated when we handle those)

If the phenotype was found during this update, we will return a **True** value, which will allow us to 

**Technical Note:** In order to check all the simple genes, we are using a [for loop](https://docs.python.org/3/tutorial/controlflow.html#for-statements), a piece of code that runs through all items in a list, and runs the same bits of code for each item in the list

In [4]:
def updateSimpleGenesWithPhenotype(bird, phenotype, simpleGenes, clearExisting = True):
    foundPhenotype = False
    
    # Check simple genes
    for gene in simpleGenes:
        # If the current gene we are checking matches the selected
        # phenotype, set the genotype in the bird equal to homozygous
        # for the trait
        if phenotype == gene['name']:
            bird[gene['name']] = gene['notation'] + '/' + gene['notation']
            foundPhenotype = True
        # If the current gene we are checking does not match the phenotype
        # we should clear the genotype for that gene
        elif clearExisting and gene['name'] in bird:
            del bird[gene['name']]

    return foundPhenotype

#### updateSimpleGenesWithPhenotype Tests
We can test this code by creating a bird with a genotype, calling the function with a different phenotype than the one that would be displayed based on the bird's genotype, and making sure that the bird's genotype is changed to match the new phenotype

Note that in this test we check that the new genotype exists, but we also check that the old genotype was deleted

In [5]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'f/f'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN updateSimpleGenes is called
found = updateSimpleGenesWithPhenotype(bird, "Bronze", colorGenes)
# THEN the bird will contain a "br/br" genotype for "Bronze"
if not bird == {'Bronze': 'br/br'}:
    raise Exception('Failed to verify simple gene converter - new phenotype not added')
# AND the bird will NOT contain a genotype for "Fake Gene"
if 'Fake Gene' in bird:
    raise Exception('Failed to verify simple gene converter - old phenotype not removed')

### Modify Multi-Allotype Genes

The second step in saving the phenotype will be to update the genotype for any genes that have multiple possible allotypes. To do this, we first clear the value for the current gene. Then we check each allotype for a specific gene to see if it matches the listed phenotype. If the name of the allotype matches the phenotype, we will update the genotype for the bird to be homozygous for the trait if it is a normal gene.  This code also accounts for sex-linked genes, so when updating the genotype for a sex-linked gene, if the bird is female it adds a "w" instead of the second copy of a homozygous allele  

If the phenotype was found during this update, we will return a True value, which will allow us to exit early when checking multiple genes at once

In [6]:
def updateGeneWithPhenotype(bird, phenotype, sex, geneName, allotypes, geneIsSexLinked = False, clearExisting = True):
    foundPhenotype = False
    
    # Reset the gene to the default wild type
    if clearExisting and geneName in bird:
        del bird[geneName]
        
    # Check all allotypes to see if they match the phenotype
    for allotype in allotypes:
        if phenotype == allotype['name']:
            if geneIsSexLinked and sex == 'Female':
                bird[geneName] = allotype['notation'] + '/w'
            else:
                bird[geneName] = allotype['notation'] + '/' + allotype['notation']
            foundPhenotype = True

    return foundPhenotype

#### updateGeneWithPhenotype Tests

In [7]:
# GIVEN a female bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/w'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN updateGeneWithPhenotype is called on a female bird for a sex-linked gene
found = updateGeneWithPhenotype(bird, "American Purple", 'Female', 'Sex-Linked Color', sexLinkedColorAllotypes, True)
# THEN the bird will contain a "Z(pl)/w" genotype for "Sex-Linked Color"
if not bird == {'Sex-Linked Color': 'Z(pl)/w'}:
    raise Exception('Failed to verify allotype gene converter - female sex-linked phenotype not added')

In [8]:
# GIVEN a male bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/Z(c)'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN updateGeneWithPhenotype is called on a male bird for a sex-linked gene
found = updateGeneWithPhenotype(bird, "American Purple", 'Male', 'Sex-Linked Color', sexLinkedColorAllotypes, True)
# THEN the bird will contain a "Z(pl)/Z(pl)" genotype for "Sex-Linked Color"
if not bird == {'Sex-Linked Color': 'Z(pl)/Z(pl)'}:
    raise Exception('Failed to verify allotype gene converter - male sex-linked phenotype not added')

In [9]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/W'}
# AND a list of pied allotypes
piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]
# WHEN updateGeneWithPhenotype is called
found = updateGeneWithPhenotype(bird, "Dark Pied", 'Female', 'Pied', piedAllotypes)
# THEN the bird will contain a "p/p" genotype for "Pied"
if not bird == {'Pied': 'p/p'}:
    raise Exception('Failed to verify allotype gene converter - phenotype not added')

### Modify Het-Allele Genes
We also want to modify the genotype for birds where the phenotype is a result of a heterogynous genotype.  Because these phenotypes may overlap, we will assume that the first one in the list whose name matches the phenotype is the correct genotype

In [10]:
def updateGenesWithHetAllelePhenotype(bird, phenotype, hetPhenotypes):
    foundPhenotype = False
        
    # Check all named het phenotypes to see if they match the phenotype
    for hetPhenotype in hetPhenotypes:
        if phenotype == hetPhenotype['name']:
            bird[hetPhenotype['geneName']] = hetPhenotype['alleles'][0] + '/' + hetPhenotype['alleles'][1]
            foundPhenotype = True

    return foundPhenotype

#### updateGeneWithHetAllelePhenotype Tests

In [11]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/W'}
# AND a list of het phenotypes
hetPhenotypes = [
    { 'name': 'Pied',      'geneName': 'Pied', 'alleles': ['W', 'p'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'p'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'W'] }
]
# WHEN updateGeneWithPhenotype is called
found = updateGenesWithHetAllelePhenotype(bird, "Pied", hetPhenotypes)
# THEN the bird will contain a "W/p" genotype for "Pied"
if not bird == {'Pied': 'W/p'}:
    raise Exception('Failed to verify allotype gene converter - phenotype not added')

### Putting It All Together

Now that we have created a handful of functions that can handle converting various phenotypes to the appropriate genotype, we can put all the smaller functions together into one larger function that checks all the different kinds of genes and allotypes to get the complete genotype for a specific phenotype

In [12]:
# This converts a phenotype to a genotype in a bird, assuming the phenotype breeds true
def savePhenotypeToBird(bird, phenotype, sex, simpleGenes, multiAllotypeGenes, 
                        sexLinkedAllotypes, multiGeneTraits, 
                        sexAndAutosomalCombos, hetTraits):
    # Update the simple genes
    foundPhenotype = updateSimpleGenesWithPhenotype(bird, phenotype, simpleGenes)
    
    # Update the sex-linked color genes
    for gene in multiAllotypeGenes:
        if updateGeneWithPhenotype(bird, phenotype, sex, gene['name'], gene['allotypes'], gene['sexLinked']):
            foundPhenotype = True

    # At this point, all genes have been cleared except for a matched phenotype
    # If we have found the matching phenotype already, we can stop here
    if foundPhenotype:
        return
                        
    # Save phenotype for het sex traits, and return if we find a phenotype
    if updateGenesWithHetAllelePhenotype(bird, phenotype, hetTraits):
        return

    # Save the phenotype data for genotypes that match multiple genes
    for trait in multiGeneTraits:
        if phenotype == trait['name']:
            # for each gene that makes up the multi-gene trait
            for geneName in trait['genesNeeded']:
                # check for that phenotype in the simple gene list and save it to the bird
                if updateSimpleGenesWithPhenotype(bird, geneName, simpleGenes, False):
                    # if we updated the genotype, that means we found a match for this phenotype
                    # and should stop looking
                    continue
    
                # check for that phenotype in the multiAllotypeGenes and save it to the bird
                foundPhenotype = False
                for gene in multiAllotypeGenes:
                    if updateGeneWithPhenotype(bird, geneName, sex, gene['name'], gene['allotypes'], gene['sexLinked'], False):
                        foundPhenotype = True
                        continue
                        
                # if we updated the genotype, that means we found a match for this phenotype
                # and should stop looking
                if foundPhenotype:
                    continue
    
                updateGenesWithHetAllelePhenotype(bird, geneName, hetTraits)
            
            break
            

#### savePhenotypeToBird Tests

In [13]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'f/f'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN savePhenotypeToBird is called
found = savePhenotypeToBird(bird, "Bronze", 'Male', colorGenes, [], [], [], [], [])
# THEN the bird will contain a "br/br" genotype for "Bronze"
if not bird == {'Bronze': 'br/br'}:
    raise Exception('Failed to verify phenotype-to-gene converter - new phenotype not added')
# AND the bird will NOT contain a genotype for "Fake Gene"
if 'Fake Gene' in bird:
    raise Exception('Failed to verify phenotype-to-gene converter - old simple phenotype not removed')

In [14]:
# GIVEN a female bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/w'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN savePhenotypeToBird is called on a female bird for a sex-linked gene
found = savePhenotypeToBird(bird, "American Purple", 'Female', [], [
    { 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }
], sexLinkedColorAllotypes, [], [], [])
# THEN the bird will contain a "Z(pl)/w" genotype for "Sex-Linked Color"
if not bird == {'Sex-Linked Color': 'Z(pl)/w'}:
    raise Exception('Failed to verify phenotype-to-gene converter - female sex-linked phenotype not added')

In [15]:
# GIVEN a male bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/Z(c)'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN savePhenotypeToBird is called on a male bird for a sex-linked gene
found = savePhenotypeToBird(bird, "American Purple", 'Male', [], [
    { 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }
], sexLinkedColorAllotypes, [], [], [])
# THEN the bird will contain a "Z(pl)/Z(pl)" genotype for "Sex-Linked Color"
if not bird == {'Sex-Linked Color': 'Z(pl)/Z(pl)'}:
    raise Exception('Failed to verify phenotype-to-gene converter - male sex-linked phenotype not added')

In [16]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/W'}
# AND a list of pied allotypes
piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]
# WHEN savePhenotypeToBird is called
found = savePhenotypeToBird(bird, "Dark Pied", 'Female', [], [
    { 'name': 'Pied', 'allotypes': piedAllotypes, 'sexLinked': False }
], piedAllotypes, [], [], [])
# THEN the bird will contain a "p/p" genotype for "Pied"
if not bird == {'Pied': 'p/p'}:
    raise Exception('Failed to verify phenotype-to-gene converter - pied allotype phenotype not added')

In [17]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/W'}
# AND a list of het phenotypes
hetPhenotypes = [
    { 'name': 'Pied',      'geneName': 'Pied', 'alleles': ['W', 'p'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'p'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'W'] }
]
# WHEN savePhenotypeToBird is called with "Pied"
found = savePhenotypeToBird(bird, "Pied", 'Female', [], [], [], [], [], hetPhenotypes)
# THEN the bird will contain a "W/p" genotype for "Pied"
if not bird == {'Pied': 'W/p'}:
    raise Exception('Failed to verify phenotype-to-gene converter - pied het phenotype not added')

In [18]:
# GIVEN an empty bird
bird = {}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'o',       'name': 'Opal' }
]
# AND a list of multi-gene phenotypes
multiGeneColors = [
    { 'name': 'Platinum',      'genes': ['br', 'o'], 'genesNeeded': ['Bronze', 'Opal' ] }
]
# WHEN savePhenotypeToBird is called with Platinum
found = savePhenotypeToBird(bird, "Platinum", 'Female', colorGenes, [], [], multiGeneColors, [], hetPhenotypes)
# THEN the bird will contain "Bronze" and "Opal"
if not bird == {'Bronze': 'br/br', 'Opal': 'o/o'}:
    raise Exception('Failed to verify phenotype-to-gene converter - phenotype not added')

{'Bronze': 'br/br', 'Opal': 'o/o'}


In [19]:
# GIVEN an empty bird
bird = {}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'o',       'name': 'Opal' }
]
# AND a list of multi-gene phenotypes
multiGeneColors = [
    { 'name': 'Platinum',      'genes': ['br', 'o'], 'genesNeeded': ['Bronze', 'Opal' ] },
    { 'name': 'Taupe',      'genes': ['br', 'o'], 'genesNeeded': ['American Purple', 'Opal' ] }
]
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN savePhenotypeToBird is called with "Taupe"
found = savePhenotypeToBird(bird, "Taupe", 'Female', colorGenes, [
    { 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }
], [], multiGeneColors, [], [])
# THEN the bird will contain "Bronze" and "Opal"
if not bird == {'Sex-Linked Color': 'Z(pl)/w', 'Opal': 'o/o'}:
    raise Exception('Failed to verify phenotype-to-gene converter - phenotype not added')

## Convert Genotype to Phenotype

Finally, we need to create a function that can convert a bird's genotype into a phenotype.  We want to account for scenarios where the genotype matches multiple different phenotypes, since some phenotypes may overlap.  (For example, the genotype for the platinum phenotype overlaps with the genotype of bronze and opal)

### Phenotypes from Simple Genes
The easiest part to convert is the simple genes.  We can simply check each simple gene, and see if the bird contains the correct genotype for that gene.  If it does, then we can update the phenotype of the bird to be the name of the gene.  We also update the list of previous phenotypes so that we can track overlapping phenotypes. (For example, a platinum bird would technically have three phenotypes - Bronze, Opal, and Platinum)  Tracking the overlapping phenotypes will let us determine if the described genotype has ever been recorded before.

**Technical Note:** We can use a [tuple](https://docs.python.org/3/library/stdtypes.html#tuples) here to return multiple different pieces of information from the function, so we can use them elsewhere.  In this case, we are returning both a string that represents the primary phenotype, and a [list](https://docs.python.org/3/library/stdtypes.html#lists) of all other previous phenotypes that we have matched to this bird

In [20]:
def getSimplePhenotypesFromBird(bird, phenotypes, simpleGenes):
    for gene in simpleGenes:
        if gene['name'] in bird:
            if bird[gene['name']] == gene['notation'] + '/' + gene['notation']:
                # Add the previous bird phenotype to the list, and change the current phenotype
                phenotypes.append(gene['name'])

    return phenotypes

#### getSimplePhenotypesFromBird Tests

In [21]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'f/f'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN getSimplePhenotypesFromBird is called
phenotypes = getSimplePhenotypesFromBird(bird, [], colorGenes)
# THEN the phenotype will be "Fake Gene"
if len(phenotypes) != 1 or not phenotypes[0] == "Fake Gene":
    raise Exception('Failed to verify simple gene phenotype converter - incorrect phenotype found')

### Phenotypes from Multi-Allotype Genes

Next, we want to check each potential allotype for more complex genes, and save any phenotypes that match to the list of found phenotypes

In [22]:
def getPhenotypesForGeneFromBird(bird, phenotypes, sex, geneName, allotypes, geneIsSexLinked = False):
    for allotype in allotypes:
        if geneName in bird and (
            # If the gene is not sex-linked, or the bird is male, check for homozygous genes
            ((not geneIsSexLinked or sex == 'Male') and bird[geneName] == allotype['notation'] + '/' + allotype['notation']) or
            # If the gene is sex linked and the bird is female, check for the female-specific variant
            ((geneIsSexLinked and sex == 'Female') and bird[geneName] == allotype['notation'] + '/w')):
            phenotypes.append(allotype['name'])

    return phenotypes

#### getPhenotypesForGeneFromBird Tests

In [23]:
# GIVEN a female bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/w'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN getPhenotypesForGeneFromBird is called on a female bird for a sex-linked gene
phenotypes = getPhenotypesForGeneFromBird(bird, [], 'Female', 'Sex-Linked Color', 
                                                            sexLinkedColorAllotypes, True)
# THEN the phenotype will be "Cameo"
if len(phenotypes) != 1 or not phenotypes[0] == "Cameo":
    raise Exception('Failed to verify allotype gene phenotype converter - incorrect phenotype found')

['Cameo']


In [24]:
# GIVEN a male bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/Z(c)'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN getPhenotypesForGeneFromBird is called on a male bird for a sex-linked gene
phenotypes = getPhenotypesForGeneFromBird(bird, [], 'Male', 'Sex-Linked Color', sexLinkedColorAllotypes, True)
# THEN the phenotype will be "Cameo"
if len(phenotypes) != 1 or not phenotypes[0] == "Cameo":
    raise Exception('Failed to verify allotype gene phenotype converter - incorrect phenotype found')

In [25]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/W'}
# AND a list of pied allotypesa
piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]
# WHEN savePhenotypeToBird is called
phenotypes = getPhenotypesForGeneFromBird(bird, [], 'Male', 'Pied', piedAllotypes)
# THEN the phenotype will be "White"
if len(phenotypes) != 1 or not phenotypes[0] == "White":
    raise Exception('Failed to verify allotype gene phenotype converter - incorrect phenotype found')

### Phenotypes from Heterozygous combinations
Some allotypes show incomplete dominance, and create a unique phenotype when they exist as heterozygous genes.  We need to loop through all these possible combinations, and add any unique phenotypes we find

In [26]:
def getHetPhenotypesFromBird(bird, phenotypes, hetPhenotypes):
    for hetPhenotype in hetPhenotypes:
        if (hetPhenotype['geneName'] in bird and 
            (bird[hetPhenotype['geneName']] == hetPhenotype['alleles'][0] + '/' + hetPhenotype['alleles'][1] or
             bird[hetPhenotype['geneName']] == hetPhenotype['alleles'][1] + '/' + hetPhenotype['alleles'][0])):
            # Add the previous bird phenotype to the list, and change the current phenotype
            phenotypes.append(hetPhenotype['name'])

    return phenotypes

#### getHetPhenotypesFromBird Tests

In [27]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/p'}
# AND a list of het phenotypes
hetPhenotypes = [
    { 'name': 'Pied',      'geneName': 'Pied', 'alleles': ['W', 'p'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'p'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'W'] }
]
# WHEN getHetPhenotypesFromBird is called
phenotypes = getHetPhenotypesFromBird(bird, [], hetPhenotypes)
# THEN the phenotype will be "Pied"
if len(phenotypes) != 1 or not phenotypes[0] == "Pied":
    raise Exception('Failed to verify het gene phenotype converter - incorrect phenotype found')

### Determining the Final Phenotype

Now that we have a bunch of functions that can determine parts of the bird's phenotype, we can put it all together to determine the final phenotype for a bird

In [28]:
def getPhenotypeFromBird(bird, sex, wildType, simpleGenes, multiAllotypeGenes, hetAllotypeGenes, multiGeneTraits):
    # Because phenotypes are not mutually exclusive, we want to track 
    # all possible phenotypes
    birdPhenotypes = []

    # We need to check all the simple genes to see if we can find a phenotype name
    birdPhenotypes = getSimplePhenotypesFromBird(bird, birdPhenotypes, simpleGenes)

    # All multi-allotype genes
    for gene in multiAllotypeGenes:
        birdPhenotypes = getPhenotypesForGeneFromBird(bird, birdPhenotypes, sex,
                                                      gene['name'], gene['allotypes'], gene['sexLinked'])

    # And all het genes
    birdPhenotypes = getHetPhenotypesFromBird(bird, birdPhenotypes, hetAllotypeGenes)

    # Once we have all possible phenotypes, we need to check if they combine
    multiGeneTraitExactMatch = False
    for phenotype in multiGeneTraits:
        matchesPhenotype = True
        # We want to make sure that all phenotypes in the genes needed list have been identified
        for gene in phenotype['genesNeeded']:
            if not gene in birdPhenotypes:
                matchesPhenotype = False
                break
                
        if matchesPhenotype:
            # We also want to check that the number of found phenotypes EXACTLY 
            multiGeneTraitExactMatch = multiGeneTraitExactMatch or len(phenotype['genesNeeded']) == len(birdPhenotypes)
            birdPhenotypes.append(phenotype['name'])

    # Birds are wild type by default, or else are the last item in the phenotypes array
    finalBirdPhenotype = birdPhenotypes[-1] if len(birdPhenotypes) > 0 else wildType
    # We also want to track whether or not this phenotype has ever been 
    # observed before (ie if we have a record of it)
    isUnknownPhenotype = False if len(birdPhenotypes) == 1 else multiGeneTraitExactMatch
    return (finalBirdPhenotype, birdPhenotypes, isUnknownPhenotype)

#### getPhenotypeFromBird Tests

In [29]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'f/f'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN getPhenotypeFromBird is called
(phenotype, otherPhenotypes, isUnknown) = getPhenotypeFromBird(bird, "Male", 'Wild Type', colorGenes, [], [], [])
# THEN the phenotype will be "Fake Gene"
if len(otherPhenotypes) != 1 or not phenotype == "Fake Gene":
    raise Exception('Failed to verify gene-to-phenotype converter - incorrect phenotype found')

In [30]:
# GIVEN a female bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/w'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN getPhenotypeFromBird is called on a female bird for a sex-linked gene
(phenotype, otherPhenotypes, isUnknown) = getPhenotypeFromBird(bird, "Female", 'Wild Type', [], 
    [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }], [], [])
# THEN the phenotype will be "Cameo"
if len(otherPhenotypes) != 1 or not phenotype == "Cameo":
    raise Exception('Failed to verify allotype gene phenotype converter - incorrect phenotype found')

In [31]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/p'}
# AND a list of het phenotypes
hetPhenotypes = [
    { 'name': 'Pied',      'geneName': 'Pied', 'alleles': ['W', 'p'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'p'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'W'] }
]
# WHEN getHetPhenotypesFromBird is called
(phenotype, otherPhenotypes, isUnknown) = getPhenotypeFromBird(bird, "Female", 'Wild Type', colorGenes, 
    [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }], hetPhenotypes, [])
# THEN the phenotype will be "Pied"
if len(otherPhenotypes) != 1 or not phenotype == "Pied":
    raise Exception('Failed to verify het gene phenotype converter - incorrect phenotype found')

In [32]:
# GIVEN a bird genome with a genotype for "Taupe"
bird = { 'Opal': 'o/o', 'Sex-Linked Color': 'Z(pl)/w' }
# AND a list of simple genes
colorGenes = [
    { 'notation': 'o',      'name': 'Opal' },
    { 'notation': 'f',      'name': 'Fake Gene' }
]
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# AND a list of multi-gene traits
multiGeneColors = [
    { 'name': 'Platinum',   'genesNeeded': ['Bronze', 'Opal' ] },
    { 'name': 'Taupe',      'genesNeeded': ['American Purple', 'Opal' ] }
]
# WHEN getHetPhenotypesFromBird is called
(phenotype, otherPhenotypes, isUnknown) = getPhenotypeFromBird(bird, "Female", 'Wild Type', colorGenes, 
    [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }], [], multiGeneColors)
# THEN the phenotype will be "Taupe"
if not phenotype == "Taupe":
    raise Exception('Failed to verify multi gene phenotype converter - incorrect phenotype found')
# AND the otherPhenotypes will have opal and purple
if not len(otherPhenotypes) == 3:
    raise Exception('Failed to verify het gene phenotype converter - other phenotypes incorrect')

['Opal', 'American Purple', 'Taupe']


# Holding

In [33]:
    

def getColorFromBird(bird, sex):
    # Birds are wild type by default
    finalBirdColor = "Wild Type"
    # Because colors are not mutually exclusive, we want to track 
    # all possible colors
    otherBirdColors = []
    
    # Set sex to default if not defined so we don't 
    # break things when we try to check the sex-linked color
    if not 'Sex-Linked Color' in bird:
        if sex == 'Male':
            bird['Sex-Linked Color'] = 'Z(WT)/Z(WT)'
        else: 
            bird['Sex-Linked Color'] = 'Z(WT)/w'
    
    # Bird color special cases
    if bird['Sex-Linked Color'] == 'Z(c)/Z(pl:c)':
        finalBirdColor = 'Cameo'
    elif bird['Sex-Linked Color'] == 'Z(pl)/Z(pl:c)':
        finalBirdColor = 'American Purple'

    # Check if bird color matches any simple colors
    for gene in colorGenes:
        if gene['name'] in bird:
            if bird[gene['name']] == gene['notation'] + '/' + gene['notation']:
                # Add the previous bird color to a list, and change the final color
                otherBirdColors.append(finalBirdColor)
                finalBirdColor = gene['name']

    # Check if bird color matches any sex-linked colors
    for allotype in sexLinkedColorAllotypes:
        if 'Sex-Linked Color' in bird and ((sex == 'Male' and bird['Sex-Linked Color'] == allotype['notation'] + '/' + allotype['notation']) or
            (sex == 'Female' and bird['Sex-Linked Color'] == allotype['notation'] + '/w')):
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = allotype['name']

    # Check bird color matches sexAndAutosomal Combo
    for color in sexAndAutosomalComboColors:
        matchesAll = True
        
        for gene in colorGenes:
            # If the current gene is part of the color
            if (gene['notation'] == color['autosomalGene'] and 
                # and the current gene is NOT in the bird
                (not gene['name'] in bird or 
                bird[gene['name']] != gene['notation'] + '/' + gene['notation'])):
                # then we know the bird does NOT have the color, and we do not 
                # have to check any other genes
                matchesAll = False
                break

        # If we did not find an autosomal gene, we don't need to check the
        # sex-linked genes
        if not matchesAll:
            continue
        
        for allotype in sexLinkedColorAllotypes:
            # If the current gene is part of the color
            if (allotype['notation'] == color['sexGene'] and 
                # and the current gene is NOT in the bird
                (not 'Sex-Linked Color' in bird or 
                 (sex == 'Male' and bird['Sex-Linked Color'] != allotype['notation'] + '/' + allotype['notation']) or
                 (sex == 'Female' and bird['Sex-Linked Color'] != allotype['notation'] + '/w'))):
                matchesAll = False
                break

        # If matchesAllGenes is still true here, we know the bird must have
            # all the needed genes for the color
        if matchesAll:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']
                

    # Check bird color matches multi-gene color
    for color in multiGeneColors:
        # Make sure all genes needed to make the color are availible in the bird
        matchesAllGenes = True
        for gene in colorGenes:
            # If the current gene is part of the color
            if (gene['notation'] in color['genes'] and 
                # and the current gene is NOT in the bird
                ( not gene['name'] in bird or 
                bird[gene['name']] != gene['notation'] + '/' + gene['notation'])):
                # then we know the bird does NOT have the color, and we do not 
                # have to check any other genes
                matchesAllGenes = False
                break

        # If matchesAllGenes is still true here, we know the bird must have
        # all the needed genes for the color
        if matchesAllGenes:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']

    for color in hetSexColors:
        if bird['Sex-Linked Color'] == color['alleles'][0] + '/' + color['alleles'][1]:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']

    return (finalBirdColor, otherBirdColors)